In [ ]:
import sys
sys.path.append("/home/lev/object-centric/edge-generation/datasets")
from preprocess_dataset import upload_data

import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torchmetrics import Accuracy, AveragePrecision, AUROC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
acc = Accuracy()
datasets_path = '/home/lev/object-centric/edge-generation/configs/datasets.cvs'
data = pd.read_csv(datasets_path, sep='\t', index_col=0)
new_cols = []
for row in data.iloc:
    print(row[0])
    name, path = row[0], row[-1]
    features, labels = upload_data(name)
    features, labels  = features.numpy(), labels.numpy()
    if features.shape[0] > 20000:
        new_cols.append([0, 0, 0, 0])
        continue
    X_train, X_val, y_train, y_val = train_test_split(features, labels,
                                                            test_size=0.3,
                                                            random_state=0)

    clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
    clf.fit(X_train, y_train)
    preds = clf.predict(X_val)
    f1_svm = np.round(f1_score(preds, y_val, average='macro'), 4)
    acc_svm = np.round(accuracy_score(preds, y_val), 4)

    clf = make_pipeline(StandardScaler(), LogisticRegression(random_state=0, max_iter=1000))
    clf.fit(X_train, y_train)
    preds = clf.predict(X_val)
    f1_lin = np.round(f1_score(preds, y_val, average='macro'), 4)
    acc_lin = np.round(accuracy_score(preds, y_val), 4)
    
    new_cols.append([f1_svm, acc_svm, f1_lin, acc_lin])
    
data = pd.concat([data,pd.DataFrame(new_cols, columns = ['f1_svm', 'acc_svm', 'f1_lin', 'acc_lin'])], axis=1)
save_path = "/home/lev/object-centric/edge-generation/configs/datasets.cvs"
data.to_csv(save_path, sep='\t')